In [9]:
team_x_dico = {
    "TOP": ["Perj0cvV5HlmD7hOSVEMIDUXiVmT-3YoI2D4CXJ7-AtpsUSN-jxMum4wo_bHAir-asl63P0KEEXXiA"],
    "JUNGLE": ["o23VSfJjadVrxBAg5_o9zHWCtIfszzpWIPHhVCOapFMIi51szTeKzYuUeCPGfpx3rO-M1tUKzVKrzw"],
    "MIDDLE": ["0QfJfN59Mz0y89RnMbb1fGt2IZCEXgdK3m_0v87dukfvhK834hV1AtptfQRxWjHkt97A2dwYLUvgQQ"],
    "BOTTOM": ["-R4MTIeBNr3ek4rdKxI-joCo1kZtDEP2VRzVSIMb2-ZaP4j836KBoMXjOs-toUXhXxYK0zeRXHiebQ"],
    "UTILITY": ["H5OVPAcfl6n-FtuLNnAMxoMoDl4A_ECqOI6Azy0tK5GRyApZpoHYvk-Zw38ho96TWldBiJ81m69FSQ"]
}

In [ ]:
import requests
from urllib.parse import quote
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

# Chargez le fichier .env
load_dotenv()

# Récupérez la clé API depuis les variables d'environnement
api_key = os.getenv("API_KEY")

if not api_key:
    raise ValueError("API key is not set. Please check your .env file.")

In [3]:

# Définissez votre clé API et l'URL de base
base_url = "https://europe.api.riotgames.com"

def get_puuid_from_riot_id_and_update_dico(game_name, tag_line, role, team_dico):
    """
    Retrieves a player's PUUID using their Riot ID (gameName and tagLine) and adds it to the team dictionary under the specified role.

    :param game_name: Riot player's name (gameName).
    :param tag_line: Tagline (e.g., EUW).
    :param role: Role to associate with the player ("TOP", "JUNGLE", etc.).
    :param team_dico: Dictionary containing roles and their PUUIDs.
    :return: None, updates the dictionary directly.
    """
    # Encode the game_name to handle spaces and special characters
    encoded_game_name = quote(game_name)

    # Construct the URL for the request
    url = f"{base_url}/riot/account/v1/accounts/by-riot-id/{encoded_game_name}/{tag_line}?api_key={api_key}"

    # Perform the GET request
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        puuid = data.get("puuid")

        if puuid:
            # Use the API to verify the PUUID
            puuid_url = f"{base_url}/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}"
            puuid_response = requests.get(puuid_url)

            if puuid_response.status_code == 200:
                # Check if the PUUID is already in the dictionary
                if role not in team_dico:
                    team_dico[role] = []

                if puuid not in team_dico[role]:
                    team_dico[role].append(puuid)
                    print(f"PUUID successfully added for role {role}: {puuid}")
                else:
                    print(f"PUUID {puuid} already exists for role {role}.")
            else:
                print(f"Error {puuid_response.status_code}: Unable to validate PUUID {puuid}.")
        else:
            print(f"Error: PUUID not found for {game_name}#{tag_line}.")
    else:
        print(f"Error {response.status_code}: Unable to retrieve PUUID for {game_name}#{tag_line}.")

In [4]:
def get_matches_with_team(team_dico, queue=None, match_count=20):
    """
    Retrieves a list of matches where all players in the dictionary are present on the same team.

    :param team_dico: Dictionary containing roles and their associated PUUIDs.
    :param queue: Queue type (e.g., 420 for Ranked Solo/Duo). Defaults to retrieving all queues.
    :param match_count: Number of matches to retrieve per player. Defaults to 20.
    :return: List of matches where all players are present on the same team.
    """
    # Collect all PUUIDs from the dictionary
    all_puuids = []
    for puuids in team_dico.values():
        if puuids:
         all_puuids.extend(puuids)

    # Ensure there are at least 5 PUUIDs for validation
    if len(all_puuids) < 5:
        return []

    # Fetch match sets for each PUUID
    match_sets = []
    for puuid in all_puuids:
        try:
            url = f"{base_url}/lol/match/v5/matches/by-puuid/{quote(puuid)}/ids?count={match_count}&api_key={api_key}"
            if queue is not None:
                url += f"&queue={queue}"

            response = requests.get(url)
            if response.status_code == 200:
                matches = set(response.json())
                match_sets.append(matches)
        except Exception:
            pass

    # Find common matches across all PUUIDs
    if len(match_sets) > 1:
        common_matches = set(match_sets[0])
        for matches in match_sets[1:]:
            common_matches &= matches
    else:
        common_matches = match_sets[0] if match_sets else set()

    # Validate participants in the common matches
    valid_matches = []
    for match_id in common_matches:
        match_url = f"{base_url}/lol/match/v5/matches/{match_id}?api_key={api_key}"
        try:
            match_response = requests.get(match_url)
            if match_response.status_code == 200:
                match_data = match_response.json()
                participants = match_data["metadata"]["participants"]

                # Ensure all players are on the same team
                team_1 = participants[:5]
                team_2 = participants[5:]

                if all(puuid in team_1 or puuid in team_2 for puuid in all_puuids):
                    valid_matches.append(match_id)
        except Exception:
            pass

    return valid_matches

In [5]:
def get_timelines_for_matches(match_ids):
    """
    Retrieve the timelines for a list of match IDs.

    :param match_ids: List of match IDs for which to retrieve timelines.
    :return: Dictionary of match IDs mapped to their timelines.
    """
    timelines = {}

    for match_id in match_ids:
        try:
            # Construct the URL for the timeline API
            url = f"{base_url}/lol/match/v5/matches/{match_id}/timeline?api_key={api_key}"
            
            # Perform the GET request
            response = requests.get(url)
            
            if response.status_code == 200:
                # Add the timeline data to the dictionary
                timelines[match_id] = response.json()
            else:
                print(f"Error {response.status_code}: Failed to retrieve timeline for match {match_id}.")
                print("Details:", response.text)
        except Exception as e:
            print(f"Error retrieving timeline for match {match_id}: {e}")

    return timelines

In [6]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_gold_difference(timelines, team_dico, mode='bar', selected_roles=None):
    """
    Visualize gold differences at 15 minutes or over time for players or the team.

    :param timelines: Dictionary of match timelines.
    :param team_dico: Dictionary of roles and their associated PUUIDs.
    :param mode: Visualization type - 'bar' for average gold diff at 15 min, 'line' for gold diff over time.
    :param selected_roles: List of roles to visualize; defaults to all roles in the dictionary.
    :return: None, displays the chart.
    """
    # Default to all roles if none are selected
    if selected_roles is None:
        selected_roles = team_dico.keys()

    # Collect gold differences for the selected roles
    gold_differences = {role: [] for role in selected_roles}
    team_gold_differences = []

    for match_id, timeline in timelines.items():
        if not timeline.get('info') or not timeline['info'].get('frames'):
            continue

        frames = timeline['info']['frames']
        participants = timeline['metadata']['participants']

        # Map roles to participant IDs
        role_to_ids = {role: participants.index(puuid) + 1 for role, puuids in team_dico.items() for puuid in puuids if role in selected_roles}

        # Process gold differences at 15 minutes
        frame_at_15 = next((frame for frame in frames if frame["timestamp"] >= 900000), None)
        if not frame_at_15:
            continue

        participant_frames = frame_at_15["participantFrames"]

        allied_gold = 0
        enemy_gold = 0

        for role, player_id in role_to_ids.items():
            opponent_id = player_id + 5 if player_id <= 5 else player_id - 5

            if str(player_id) in participant_frames and str(opponent_id) in participant_frames:
                player_gold = participant_frames[str(player_id)]["totalGold"]
                opponent_gold = participant_frames[str(opponent_id)]["totalGold"]

                gold_diff = player_gold - opponent_gold
                gold_differences[role].append(gold_diff)

                # Accumulate team gold for allied team
                if player_id <= 5:  # Allied team
                    allied_gold += player_gold
                    enemy_gold += opponent_gold

        team_gold_differences.append(allied_gold - enemy_gold)

    # Visualization
    if mode == 'bar':
        # Calculate averages for bar chart
        averages = {role: np.mean(diffs) if diffs else 0 for role, diffs in gold_differences.items()}
        roles = list(averages.keys())
        values = list(averages.values())

        plt.bar(roles, values)
        plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
        plt.title("Average Gold Difference at 15 Minutes")
        plt.ylabel("Gold Difference")
        plt.xlabel("Roles")
        plt.show()

    elif mode == 'line':
        # Calculate average gold difference over time for the team
        time_points = range(15)
        team_gold_diff_time_series = []

        for match_id, timeline in timelines.items():
            if not timeline.get('info') or not timeline['info'].get('frames'):
                continue

            frames = timeline['info']['frames']
            match_gold_diff = []

            for frame in frames:
                timestamp = frame["timestamp"]
                if timestamp > 900000:  # Stop at 15 minutes
                    break

                participant_frames = frame["participantFrames"]
                allied_gold = 0
                enemy_gold = 0

                for role, player_id in role_to_ids.items():
                    if str(player_id) in participant_frames:
                        player_gold = participant_frames[str(player_id)]["totalGold"]
                        allied_gold += player_gold

                        opponent_id = player_id + 5 if player_id <= 5 else player_id - 5
                        if str(opponent_id) in participant_frames:
                            enemy_gold += participant_frames[str(opponent_id)]["totalGold"]

                match_gold_diff.append(allied_gold - enemy_gold)

            if len(match_gold_diff) >= len(time_points):
                team_gold_diff_time_series.append(match_gold_diff[:15])

        avg_gold_diff = np.mean(team_gold_diff_time_series, axis=0)

        plt.plot(time_points, avg_gold_diff, label="Team Gold Diff")
        plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
        plt.title("Gold Difference Over Time (First 15 Minutes)")
        plt.ylabel("Gold Difference")
        plt.xlabel("Time (minutes)")
        plt.legend()
        plt.show()

In [7]:
def plot_filled_gold_difference(timelines, team_dico):
    """
    Plot a filled line chart showing gold surplus/deficit over time for the team.

    :param timelines: Dictionary of match timelines.
    :param team_dico: Dictionary of roles and their associated PUUIDs.
    :return: None, displays the chart.
    """
    # Prepare data for the plot
    team_gold_diff_time_series = []

    for match_id, timeline in timelines.items():
        if not timeline.get('info') or not timeline['info'].get('frames'):
            continue

        frames = timeline['info']['frames']
        participants = timeline['metadata']['participants']

        # Map roles to participant IDs
        role_to_ids = {role: participants.index(puuid) + 1 for role, puuids in team_dico.items() for puuid in puuids}

        match_gold_diff = []
        for frame in frames:
            timestamp = frame["timestamp"]
            if timestamp > 900000:  # Stop at 15 minutes
                break

            participant_frames = frame["participantFrames"]
            allied_gold = 0
            enemy_gold = 0

            for role, player_id in role_to_ids.items():
                if str(player_id) in participant_frames:
                    player_gold = participant_frames[str(player_id)]["totalGold"]
                    allied_gold += player_gold

                    opponent_id = player_id + 5 if player_id <= 5 else player_id - 5
                    if str(opponent_id) in participant_frames:
                        enemy_gold += participant_frames[str(opponent_id)]["totalGold"]

            match_gold_diff.append(allied_gold - enemy_gold)

        if match_gold_diff:
            team_gold_diff_time_series.append(match_gold_diff)

    # Calculate the average gold difference over time
    avg_gold_diff = np.mean(team_gold_diff_time_series, axis=0)

    # Plot the filled line chart
    time_points = range(len(avg_gold_diff))
    plt.fill_between(time_points, avg_gold_diff, 0, where=(avg_gold_diff > 0), interpolate=True, color='green', alpha=0.6, label='Surplus (Team Ahead)')
    plt.fill_between(time_points, avg_gold_diff, 0, where=(avg_gold_diff < 0), interpolate=True, color='red', alpha=0.6, label='Deficit (Team Behind)')

    plt.plot(time_points, avg_gold_diff, color='black', linewidth=1.5, label='Gold Difference')
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.8)
    plt.title("Gold Difference Over Time (Surplus/Deficit)")
    plt.ylabel("Gold Difference")
    plt.xlabel("Time (minutes)")
    plt.legend()
    plt.show()

In [ ]:
#get_puuid_from_riot_id_and_update_dico("PoroKamï","KLED","JUNGLE",team_x_dico)
matches_with_team=get_matches_with_team(team_x_dico, queue=440, match_count=20)
timelines=get_timelines_for_matches(matches_with_team)
plot_filled_gold_difference(timelines,team_x_dico)
visualize_gold_difference(timelines, team_x_dico, mode='bar')
